In [1]:
from libs.alibaba.alibaba import Alibaba
from libs.alibaba.p4p import P4P
from libs.json import JSON

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from bs4 import BeautifulSoup
from pyquery import PyQuery as pq

import json
import requests
import redis
import pendulum
import arrow
import imp
import time
import re

import types
import traceback 
import threading

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-software-rasterizer')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-logging')
chrome_options.add_argument('--disable-infobars')
chrome_options.add_argument('--ignore-certificate-errors')
browser = webdriver.Chrome(chrome_options=chrome_options)

In [3]:
broker_url = 'redis://localhost:6379/0'
market = JSON.deserialize('.', 'storage', 'markets.json')['Eyelashes']
p4p = P4P(market, market['lid'], market['lpwd'],broker_url=broker_url, headless=False, browser=browser)

. storage markets.json
.\storage\markets.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_recording.json
E:\glitter\Eyelashes_config\p4p_keywords_list_recording.json was deserialized!
E:\glitter\Eyelashes_config [] p4p_keywords_list_monitor.json
E:\glitter\Eyelashes_config\p4p_keywords_list_monitor.json was deserialized!


In [ ]:
p4p.crawl = types.MethodType(crawl, p4p)

In [ ]:
p4p.save_crawling_result = types.MethodType(save_crawling_result, p4p)

In [ ]:
p4p.find_prices_and_sponsors = types.MethodType(find_prices_and_sponsors, p4p)

In [ ]:
p4p.find_sponsors = types.MethodType(find_sponsors, p4p)

In [ ]:
p4p.browser.set_window_size(1920, 1200)

In [ ]:
p4p.browser.save_screenshot('screenie.png')

In [4]:
group = '关注词_2'

In [ ]:
p4p.load_url()
p4p.switch_to_group(group)

In [ ]:
p4p.crawl(group=group)

In [6]:
p4p.set_sub_budget(85)

In [13]:
p4p.unset_sub_budget()

In [12]:
p4p.monitor(group=group, sub_budget_limited=True)

Task Monitor start to run with group="关注词_2"
switch to group: 关注词_2
index: 0 8 > clear band lashes >  >>>>>> successful end 
index: 1 8 > clear band mink eyelashes >  >>>>>> successful end 
index: 2 8 > eyelash extensions private label >  >>>>>> successful end 
index: 3 8 > faux mink eyelashes >  >>>>>> successful end 
index: 4 8 > mink fur eyelashes > 不需要改变价格 >  >>>>>> successful end 
index: 5 8 > natural eyelashes >  >>>>>> successful end 
index: 6 8 > silk eyelashes private label >  >>>>>> successful end 
index: 7 8 > thin band eyelashes > 不需要改变价格 >  >>>>>> successful end 
next_page: False
next_page: False


In [ ]:
p4p.turn_all_off(group=group)